<a href="https://colab.research.google.com/github/notwld/chatbot/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import nltk
nltk.download("punkt")

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import tensorflow as tf
import numpy as np
import random,json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# Using google colab's files method to upload json file which contains the data 

from google.colab import files
files.upload()


Saving intents.json to intents.json


{'intents.json': b'{\r\n        "intents": [\r\n                {\r\n                        "tag": "greeting",\r\n                        "patterns": [\r\n                                "Hi",\r\n                                "How are you",\r\n                                "Is anyone there?",\r\n                                "Hello",\r\n                                "Good day"\r\n                        ],\r\n                        "responses": [\r\n                                "Hello, thanks for visiting",\r\n                                "Good to see you again",\r\n                                "Hi there, how can I help?"\r\n                        ],\r\n                        "context_set": ""\r\n                },\r\n                {\r\n                        "tag": "goodbye",\r\n                        "patterns": [\r\n                                "Bye",\r\n                                "See you later",\r\n                                "Goodbye"\r\n     

In [8]:
with open("intents.json",'r') as data:
  intents = json.load(data)

In [ ]:
intents

In [10]:
words = []
classes = []
documents = []
ignore = ["?"]

In [11]:
# data preprocessing: looping through json file which is saved in variable 
# intents and tokenizing the patterns, extract word with it's tag and saving 
# them in the list.

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word = nltk.word_tokenize(pattern)
    words.extend(word)

    documents.append((word,intent["tag"]))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])



In [12]:
words = [stemmer.stem(word.lower()) for word in words if word not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))


In [13]:
print("Total documents:", len(documents))
print("Total classes:", len(classes))
print("Total unique words:", len(words))

Total documents: 27
Total classes: 8
Total unique words: 52


In [14]:
# creating lists for training the data

train = [] # f(x)
output = [] # f(y)

output_empty = [0] * len(classes)

In [15]:
#test

documents[0]

(['Hi'], 'greeting')

In [16]:
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if word in pattern_words else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])]=1

  train.append([bag,output_row])

random.shuffle(train)

train = np.array(train)

x = list(train[:,0])
y = list(train[:,1])


<ipython-input-16-719e020afe1d>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(train)


In [17]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(y[0]),activation="softmax"))

model.compile(tf.keras.optimizers.Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

In [18]:
model.fit(np.array(x),np.array(y),epochs=100,batch_size=8,verbose=1)
model.save("model.pkl")

Epoch 1/100
4/4 [==============================] - 1s 6ms/step - loss: 2.0806 - accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0791 - accuracy: 0.1852
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0785 - accuracy: 0.1852
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0778 - accuracy: 0.1852
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0776 - accuracy: 0.1852
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0771 - accuracy: 0.1852
Epoch 7/100
4/4 [==============================] - 0s 5ms/step - loss: 2.0762 - accuracy: 0.1852
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0758 - accuracy: 0.1852
Epoch 9/100
4/4 [==============================] - 0s 6ms/step - loss: 2.0755 - accuracy: 0.1852
Epoch 10/100
4/4 [==============================] - 0s 5ms/step - loss: 2.0746 - accuracy: 0.1852
Epoch 11/100
4/4 [=======

In [27]:
# prediction

import pickle
pickle.dump({"words":words,"classes":classes},open("training_data","wb"))



In [28]:
from keras.models import load_model
model = load_model("model.pkl")

In [29]:
with open("training_data","rb") as data:
  data = pickle.load(data)
  words = data["words"]
  classes = data["classes"]


In [30]:
with open("intents.json",'r') as data:
  intents = json.load(data)

In [31]:
def cleanUp(sentence):
  words = nltk.word_tokenize(sentence)
  return [stemmer.stem(word.lower()) for word in words]

In [32]:
def bow(sentence, words):
  sentence = cleanUp(sentence)
  bag = [0] * len(words)
  for s in sentence:
    for i,w in enumerate(words):
      if w==s:
        bag[i] = 1

  return (np.array(bag))